## BERT

In [ ]:
import torch

from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
model = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')

# tokenizer = AutoTokenizer.from_pretrained("ohshimalab/bert-base-minpaku")
# model = AutoModelForMaskedLM.from_pretrained("ohshimalab/bert-base-minpaku")

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
# model.to(device)

In [ ]:
# initialize dictionary to store tokenized sentences
tokens = {'input_ids': [], 'attention_mask': []}

for sentence in documents:
    # encode each sentence and append to dictionary
    new_tokens = tokenizer.encode_plus(sentence, max_length=128,
                                       truncation=True, padding='max_length',
                                       return_tensors='pt')

    tokens['input_ids'].append(new_tokens['input_ids'][0])
    tokens['attention_mask'].append(new_tokens['attention_mask'][0])

# reformat list of tensors into single tensor
tokens['input_ids'] = torch.stack(tokens['input_ids'])
tokens['attention_mask'] = torch.stack(tokens['attention_mask'])

In [ ]:
outputs = model(**tokens)

embeddings = outputs.last_hidden_state
attention_mask = tokens['attention_mask']
mask = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
masked_embeddings = embeddings * mask
summed = torch.sum(masked_embeddings, 1)
summed_mask = torch.clamp(mask.sum(1), min=1e-9)

mean_pooled = summed / summed_mask


: 

In [ ]:
# store mean pooled in other variable
embeddings_book = mean_pooled

In [ ]:
# store locally
embeddings_book = embeddings_book.cpu().detach().numpy()

In [ ]:
torch.save(mean_pooled, 'book_embedded.t')

In [ ]:
#  = mean_pooled.cpu()
# mean_pooled = mean_pooled.detach().numpy().cpu()

# calculate
cosine_similarity(
    [mean_pooled[0]],
    mean_pooled[1:]
).argmax()

1

In [ ]:
evaluation_corpus = evaluation_metadata.replace(np.nan,'',regex=True)
documents_evaluation = list(evaluation_corpus['data'])

In [ ]:
# initialize dictionary to store tokenized sentences
tokens_eval = {'input_ids': [], 'attention_mask': []}

for sentence in documents_evaluation:
    # encode each sentence and append to dictionary
    new_tokens = tokenizer.encode_plus(sentence, max_length=128,
                                       truncation=True, padding='max_length',
                                       return_tensors='pt')
    new_tokens.to('cpu')

    tokens_eval['input_ids'].append(new_tokens['input_ids'][0])
    tokens_eval['attention_mask'].append(new_tokens['attention_mask'][0])

# reformat list of tensors into single tensor
tokens_eval['input_ids'] = torch.stack(tokens_eval['input_ids'])
tokens_eval['attention_mask'] = torch.stack(tokens_eval['attention_mask'])

In [ ]:
outputs_eval = model(**tokens_eval)

In [ ]:
embeddings = outputs_eval.last_hidden_state

In [ ]:
attention_mask = tokens_eval['attention_mask']

In [ ]:
mask = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()

In [ ]:
masked_embeddings = embeddings * mask

In [ ]:
summed = torch.sum(masked_embeddings, 1)

In [ ]:
summed_mask = torch.clamp(mask.sum(1), min=1e-9)

In [ ]:
mean_pooled = summed / summed_mask

In [ ]:
evaluation = mean_pooled

In [ ]:
evaluation = evaluation.cpu().detach().numpy()

In [ ]:
torch.save(evaluation, '/content/evaluation_embeddings.t')

In [ ]:
evaluation.shape, embeddings_book.shape

(torch.Size([116, 768]), torch.Size([185, 768]))

In [ ]:
torch.cuda.empty_cache()

In [ ]:
embeddings_book = embeddings_book.detach().numpy()

In [ ]:
device.reset()

AttributeError: ignored

In [ ]:
!nvidia-smi

Mon Jun 12 19:06:47 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    57W / 400W |  40507MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
torch.cuda.clear_cache()

AttributeError: ignored

In [ ]:
# evaluate
predictions = []
pred_y_scores = []

for i in range(evaluation.shape[0]):
    embedding = evaluation[i]

    p = cosine_similarity(
        [embedding],
        embeddings_book).argmax()

    predictions += [p]

In [ ]:
from sklearn.metrics import accuracy_score, top_k_accuracy_score

print('accuracy of the BERT model:')
print(accuracy_score(list(ground_truth['page']), predictions, normalize=True))

# print('top-k accuracy of the BM25 model:')
# print(top_k_accuracy_score(ground_truth['page'], np.array(pred_y_scores), k=10,normalize=True, labels=list(range(0,185))))

accuracy of the BERT model:
0.008620689655172414
